In [1]:
import torch
# from openmmtorch import TorchForce
import os

import yaml
from pathlib import Path


import numpy as np
from openmm.app import *
from openmm import *
from openmm.unit import *


from sys import stdout

from simutils import ForceReporter, ForceModelConvert
from maputils import EquiValReporter

from traj_reporter import TrajectoryReporter

/home/enere@usi.ch/miniconda3/envs/fmartip2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import yaml
import numpy as np
from pathlib import Path
from cgmap.utils import DataDict
from cgmap.mapping import Mapper

from typing import Dict

from aggforce import linearmap as lm
from aggforce import agg as ag
from aggforce import constfinder as cf

In [3]:
pdb = PDBFile('StartingStructures/chignolin_frames0.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = NoseHooverIntegrator(310*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('../datasets/VacumeSims/testoutput.pdb', 500))
# simulation.reporters.append(XTCReporter('../datasets/VacumeSims/testoutput.xtc', 50, append=False))
simulation.reporters.append(TrajectoryReporter('../datasets/VacumeSims/testoutput.dat', 500, step=True, potentialEnergy=True,
         kineticEnergy=True, temperature=True, time=True, totalEnergy=True))
simulation.reporters.append(ForceReporter('../datasets/VacumeSims/testforces.dat', 500))

simulation.step(5000)
forces = simulation.reporters[2].getForces()
state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)

In [4]:
trrreport = simulation.reporters[1]
trrreport.dump('../datasets/VacumeSims/testoutput.pdb', '../datasets/VacumeSims/testoutput.trr')

In [5]:
forces.shape

(10, 166, 3)

In [6]:
config = {
    'mapping': 'martini3',
    'input': 'StartingStructures/chignolin_frames0.pdb',
    'inputtraj': ['../datasets/VacumeSims/testoutput.trr'],
    'selection': 'protein',
    'output': 'pro.gro',
    'outputtraj': 'xtc',
}

# Map atoms to beads (only protein for now)
mapping = Mapper(config)
mapping.map()
dataset = mapping.dataset

# force_set = forces[:,162]



/home/enere@usi.ch/miniconda3/envs/fmartip2/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/home/enere@usi.ch/miniconda3/envs/fmartip2/lib/python3.11/site-packages/MDAnalysis/coordinates/XDR.py:240: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "
Missing TYR_H1 in mapping file
Missing TYR_H2 in mapping file
Missing TYR_H3 in mapping file
Missing TYR_OXT in mapping file


In [7]:
equvalrep = EquiValReporter(dataset=dataset)

In [8]:
equvalrep.bondMapper(config_file_path="../fmartip/ff/bond_config.yaml")

Side Chain Bond Matrix Shape (54, 2)
All bonds Mat Shape (72, 2)
Bonds to be removed [] Bonds to be replaced with correct value []
Number of issued found: 0
Number of issues remain: 0


True

In [9]:
equvalrep.angleMapper(conf_angles_path="config files/config.naive.per.bead.type.angles.yaml")

True

In [10]:
equvalrep.improperDihedralMapper()

[21]
[[21, 22, 23, 24, 25, 26]]
[ 0  5 27]
[[21, 22, 23, 24, 25, 26], [0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [27, 28, 29, 30, 31]]
[21, 22, 23, 24, 25, 26]
[22, 23, 25, 26]
[-0.21052643 -0.10127469 -0.07344736] [ 0.16615803 -0.30515071  0.11204046] [-0.16615803  0.30515071 -0.11204046] [-0.19165706 -0.09111921 -0.07048485]
[0, 1, 2, 3, 4]
[1 2 3 4]
[-0.22781514 -0.09302441 -0.05430636] [ 0.16830585 -0.14559914  0.12547652] [-0.16830585  0.14559914 -0.12547652] [-0.23435209 -0.09407391 -0.05366509]
[5, 6, 7, 8, 9]
[6 7 8 9]
[-0.19010701  0.16260578  0.03153797] [ 0.08393348 -0.04469527 -0.23965578] [-0.08393348  0.04469527  0.23965578] [-0.18983577  0.1664407   0.03739517]
[27, 28, 29, 30, 31]
[28 29 30 31]
[-0.01484138  0.12076151 -0.22153118] [-0.20368396 -0.13613173  0.07925915] [ 0.20368396  0.13613173 -0.07925915] [-0.01519755  0.13481999 -0.21627785]
[-3.12723488 -3.130571   -3.12549178 -3.07878542]
['TRP_SC1' 'TRP_SC2' 'TRP_SC5' 'TRP_SC4'] [0] [22] [0 1 2]
['TYR_SC1' 'TYR_SC2' 'TYR_SC

True

In [11]:
equvalrep.beadChargeMapper()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


True

In [12]:
equvalrep.reportEquiVals(reportPath='')

config.dihedrals.yaml successfully saved!


In [13]:
equvalrep.getDataset()

{'num_residues': 10,
 'resnames': array(['TYR', 'TYR', 'ASP', 'PRO', 'GLU', 'THR', 'GLY', 'THR', 'TRP',
        'TYR'], dtype='<U3'),
 'residcs': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'resnumbers': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'num_atoms': 165,
 'atom_pos': array([[[3.37981653, 2.28659248, 2.47441721],
         [       nan,        nan,        nan],
         [3.26928663, 2.23325443, 2.55660582],
         ...,
         [2.88135242, 2.68331504, 2.4940877 ],
         [2.87560201, 2.57427192, 2.68063641],
         [2.77202225, 2.5454638 , 2.67037296]],
 
        [[3.38977838, 2.27687263, 2.46249485],
         [       nan,        nan,        nan],
         [3.28422642, 2.21805501, 2.54786849],
         ...,
         [2.87453437, 2.66593313, 2.52349854],
         [2.87246132, 2.5406127 , 2.69970703],
         [2.76885796, 2.51220942, 2.68859005]],
 
        [[3.39724874, 2.28801537, 2.46839619],
         [       nan,        nan,        nan],
         [3.29496598, 2.2127

In [ ]:
config_file = "../fmartip/ff/bond_config.yaml"
conf: Dict = yaml.safe_load(Path(config_file).read_text())

Bond_for_bead_dict = {}

bonds_mat = []
frame = 0

for root_bead, leaf_beads in conf.items():
    
    
    if root_bead in dataset["bead_idnames"]:
        bead_types = leaf_beads.split()
        Bond_for_bead_dict.update({root_bead : np.zeros(len(bead_types))})
        beads_to_bond = np.where(root_bead == dataset["bead_idnames"])

        for i in bead_types:
            dist_mat = []

            if i == "Skip":
                continue

            beads_to_consider = np.where(i == dataset["bead_idnames"])

            # Get the bead that is closest to the one that have been chosen w.r.t. the positions matrix
            
            root_pos = dataset["bead_pos"][frame][beads_to_bond]
            leaf_pos = dataset["bead_pos"][frame][beads_to_consider] 

            for bead_pos in root_pos:
                dist_mat.append(np.linalg.norm(leaf_pos - bead_pos, axis=1))

            dist_mat = np.asarray(dist_mat)
            
            if i == root_bead :
                # Getting diagonal indices
                ind = np.diag_indices_from(dist_mat)

                # Giving the diagonals inf value to avoid self assigned bonds in the computation
                dist_mat[ind] = np.inf

            # assign beads to be bonded with minimum distance
            leafs_to_bond = dist_mat.argmin(axis=1)

            # Generate the matrix of side chain bonds
            for index in range(len(beads_to_bond[0])):
                bonds_mat.append([beads_to_bond[0][index] , beads_to_consider[0][leafs_to_bond[index]] ])

bonds_mat = np.asarray(bonds_mat)

print("Side Chain Bond Matrix Shape" , bonds_mat.shape)

BB_beads_to_bond = np.array((0))
BB_bonds_mat = []
initial = True
for root_bead, leaf_beads in conf.items():
    
    if root_bead in dataset["bead_idnames"]:
        
        # Filter Backbone Beads
        if "_BB" in root_bead:    
            
            if initial == True:
                initial = False
                BB_beads_to_bond = np.asarray(np.where(root_bead == dataset["bead_idnames"]))
            else:
                temp = np.asarray(np.where(root_bead == dataset["bead_idnames"]))
                BB_beads_to_bond = np.concatenate((BB_beads_to_bond, temp), axis = 1)  

BB_beads_to_bond = sorted(BB_beads_to_bond[0])

for i in range(len(BB_beads_to_bond)-1):
    BB_bonds_mat.append([BB_beads_to_bond[i],BB_beads_to_bond[i+1]])
    BB_bonds_mat.append([BB_beads_to_bond[i+1],BB_beads_to_bond[i]])


# Combine SC and BB bonds in one Matrix
all_bonds_mat = np.append(bonds_mat, BB_bonds_mat, axis=0)
print("All bonds Mat Shape",all_bonds_mat.shape)

# Sort the Bonds 
sorted_all_bonds_mat = all_bonds_mat[all_bonds_mat[:, 0].argsort()]

pos_bond_mat = dataset["bead_pos"][:,sorted_all_bonds_mat,:]
# print(pos_bond_mat.shape)
# print((pos_bond_mat[:,:,0,:] - pos_bond_mat[:,:,1,:]).shape)
dist_bond_mat = np.linalg.norm(pos_bond_mat[:,:,0,:] - pos_bond_mat[:,:,1,:], axis=2)
# print(dist_bond_mat.shape)
dist_mean_mat = np.average(dist_bond_mat, axis=0).reshape(-1,1)
# print(dist_mean_mat.shape)
dist_mean_mat = np.append(sorted_all_bonds_mat.reshape(-1,2), dist_mean_mat, axis=1)

dist_mean_mat = np.append(dataset["bead_idnames"][sorted_all_bonds_mat].reshape(-1,2), dist_mean_mat, axis=1)


bead_bond_dict = {}
for root_bead, leaf_bead, index_root, index_leaf, bond_dist in dist_mean_mat:
    # print(root_bead, leaf_bead)
    if root_bead in bead_bond_dict:
        flag = False

        for i in range(len(bead_bond_dict[root_bead])):
            if leaf_bead in str(bead_bond_dict[root_bead][i]):
                bead_bond_dict[root_bead][i].append(bond_dist)
                # print(root_bead, leaf_bead)
                flag = True

        if flag == False:
            bead_bond_dict[root_bead].append([leaf_bead,bond_dist])
            flag = False

    elif root_bead not in bead_bond_dict :
        bead_bond_dict.update({root_bead : []})
        bead_bond_dict[root_bead].append([leaf_bead,bond_dist])


bead_bond_dict_mean = {}
for root_bead, leaf_bead, index_root, index_leaf, bond_dist in dist_mean_mat:
    bead_bond_dict_mean[root_bead] =  {}
    for i in range(len(bead_bond_dict[root_bead])):
        # print(np.asarray(bead_bond_dict["SER_BB"][i][0]))
        temp = list(map(float, bead_bond_dict[root_bead][i][1:]))
        temp = np.mean(temp)
        key = bead_bond_dict[root_bead][i][0]
        bead_bond_dict_mean[root_bead].update({key : temp})


num_issues = 0
to_remove = []
to_replace = []
for root_bead in bead_bond_dict_mean :
    for leaf_bead in bead_bond_dict_mean[root_bead]:
        # print(root_bead,leaf_bead)
        if leaf_bead not in bead_bond_dict_mean or root_bead not in bead_bond_dict_mean[leaf_bead] :
            print("Between",root_bead,"and",leaf_bead,"no bond found")
            to_remove.append([root_bead,leaf_bead])
            num_issues += 1
            continue

        elif np.abs(bead_bond_dict_mean[root_bead][leaf_bead] - bead_bond_dict_mean[leaf_bead][root_bead]) > 0.0001:
            print("Distance mismatch found!")
            print(root_bead,"to",leaf_bead,"mean distance = ",bead_bond_dict_mean[root_bead][leaf_bead],"vs",leaf_bead,"to",root_bead,"mean distance = ",bead_bond_dict_mean[leaf_bead][root_bead])
            num_issues += 1
            to_replace.append([root_bead,leaf_bead])
            print("Difference mean distance =",np.abs(bead_bond_dict_mean[root_bead][leaf_bead] - bead_bond_dict_mean[leaf_bead][root_bead]))

print("Bonds to be removed",to_remove,"Bonds to be replaced with correct value",to_replace)
print("Number of issued found:",num_issues)

# for bead in to_remove:
#     try:
#         del bead_bond_dict_mean[bead[0]][bead[1]]
#         num_issues -= 1
#     except:
#         print(bead,"removed")
#     try:
#         del bead_bond_dict_mean[bead[1]][bead[0]]
#         num_issues -= 1
#     except:
#         print(bead,"removed")

for bead in to_replace:
    if(bead_bond_dict_mean[bead[0]][bead[1]] > bead_bond_dict_mean[bead[1]][bead[0]]):
        bead_bond_dict_mean[bead[0]][bead[1]] = bead_bond_dict_mean[bead[1]][bead[0]]
        num_issues -= 1
        print(bead,"replaced")
    else:
        bead_bond_dict_mean[bead[1]][bead[0]] = bead_bond_dict_mean[bead[0]][bead[1]]
        num_issues -= 1
        

    
print("Number of issues remain:",num_issues)

config_bonds = []
conf_dict = {}
for root_bead in bead_bond_dict_mean:
    for leaf_bead in bead_bond_dict_mean[root_bead]:
        if (str(root_bead + '-' + leaf_bead) and str(leaf_bead + '-' + root_bead)) not in conf_dict.keys():
            conf_dict[str(root_bead + '-' + leaf_bead)] = bead_bond_dict_mean[root_bead][leaf_bead]
            config_bonds.append("{}-{} : {}".format(root_bead,leaf_bead, bead_bond_dict_mean[root_bead][leaf_bead]))




Side Chain Bond Matrix Shape (54, 2)
All bonds Mat Shape (72, 2)
Bonds to be removed [] Bonds to be replaced with correct value []
Number of issued found: 0
Number of issues remain: 0


In [ ]:
conf = conf_dict
dataset['bond_indices'] = sorted_all_bonds_mat

In [ ]:
bonds = dataset["bond_indices"]
angle_beads = []
angles = {}
frame = 0
for index in range(bonds.max()+1):
    i, j= np.where(bonds == index)
    angle_beads.append([index,bonds[i,np.abs((j-1)%2)]])
    # angle_beads.append([index, np.delete(bonds[i][:,1], np.where(bonds[i][:,1] == index))])
    # print(angle_beads[index])
    for k in range(len(angle_beads[index][1])):
        pair  = (k+1)%len(angle_beads[index][1])
        bond = [angle_beads[index][1][k],angle_beads[index][0],angle_beads[index][1][pair]]
        if bond[0] == bond[2]:
            continue
        else:
            # print(bond)
            #insert 3d angle calculation
            vector1 = dataset["bead_pos"][:,angle_beads[index][1][k]] - dataset["bead_pos"][:,angle_beads[index][0]]
            vector2 = dataset["bead_pos"][:,angle_beads[index][1][pair]] - dataset["bead_pos"][:,angle_beads[index][0]]
            # print(vector1)
            # print(np.linalg.norm(vector1, axis=1))
            unit_vec1 = vector1[:] / np.linalg.norm(vector1, axis=-1)[:,None]
            unit_vec2 = vector2[:] / np.linalg.norm(vector2, axis=-1)[:,None]
            # print(unit_vec2.shape , unit_vec1.shape)
            # angle = np.arccos(np.sum(unit_vec1 * unit_vec2, axis=-1))
            angle = np.average(np.arccos(np.sum(unit_vec1 * unit_vec2, axis=-1)))
            # print(angle)
            # print(vector1[1]/(np.linalg.norm(vector1[1])))
            # # print(vector1, vector2, angle)


            if bond[0] < bond[2]:
                angles[str(bond[0]) + "-" + str(bond[1]) + "-" + str(bond[2])] = angle
            else:
                angles[str(bond[2]) + "-" + str(bond[1]) + "-" + str(bond[0])] = angle

            # print(dataset['bead_names'][bond[0]], dataset['bead_names'][bond[1]], dataset['bead_names'][bond[2]])

            # if bond[0] < bond[2]:
            #     angles.update({dataset['bead_names'][bond[0]] : {dataset['bead_names'][bond[1]] : {dataset['bead_names'][bond[2]] : [angle]}}})
            # else:
            #     angles.update({dataset['bead_names'][bond[2]] : {dataset['bead_names'][bond[1]] : {dataset['bead_names'][bond[0]] : [angle]}}})

        


In [ ]:
angle_indices = np.asanyarray([list(map(int, item.split('-'))) for item in angles.keys()])
dataset['bead_idnames'][angle_indices]

array([['TYR_SC1', 'TYR_BB', 'TYR_BB'],
       ['TYR_BB', 'TYR_SC1', 'TYR_SC3'],
       ['TYR_SC2', 'TYR_SC1', 'TYR_SC3'],
       ['TYR_SC1', 'TYR_SC2', 'TYR_SC4'],
       ['TYR_SC2', 'TYR_SC4', 'TYR_SC3'],
       ['TYR_SC1', 'TYR_SC3', 'TYR_SC4'],
       ['TYR_BB', 'TYR_BB', 'ASP_BB'],
       ['TYR_SC1', 'TYR_BB', 'ASP_BB'],
       ['TYR_BB', 'TYR_BB', 'TYR_SC1'],
       ['TYR_BB', 'TYR_SC1', 'TYR_SC2'],
       ['TYR_SC2', 'TYR_SC1', 'TYR_SC3'],
       ['TYR_BB', 'TYR_SC1', 'TYR_SC3'],
       ['TYR_SC1', 'TYR_SC2', 'TYR_SC4'],
       ['TYR_SC2', 'TYR_SC4', 'TYR_SC3'],
       ['TYR_SC1', 'TYR_SC3', 'TYR_SC4'],
       ['TYR_BB', 'ASP_BB', 'PRO_BB'],
       ['ASP_SC1', 'ASP_BB', 'PRO_BB'],
       ['ASP_BB', 'PRO_BB', 'GLU_BB'],
       ['ASP_BB', 'PRO_BB', 'PRO_SC1'],
       ['PRO_SC1', 'PRO_BB', 'GLU_BB'],
       ['PRO_BB', 'GLU_BB', 'THR_BB'],
       ['PRO_BB', 'GLU_BB', 'GLU_SC1'],
       ['GLU_SC1', 'GLU_BB', 'THR_BB'],
       ['GLU_BB', 'THR_BB', 'GLY_BB'],
       ['THR_SC1', 'THR_BB

In [ ]:
conf_angles: list = yaml.safe_load(Path("config files/config.naive.per.bead.type.angles.yaml").read_text())
angle_dic = conf_angles
angle_array = np.asanyarray([values for values in angles.values()])
print(angle_array)
for i in dataset['bead_idnames'][angle_indices]:
    rows = np.where(dataset['bead_idnames'][angle_indices[:,1]].__eq__(i[1]))[0]
    print(rows)
    print(angle_indices[rows,1])
    print(dataset['bead_idnames'][angle_indices[rows]])
    # print(dataset['bead_names'][angle_indices[rows,2]])
    sides = []
    sides = np.union1d(np.intersect1d(np.where(dataset['bead_idnames'][angle_indices[rows,0]].__eq__(i[2]))[0],np.where(dataset['bead_idnames'][angle_indices[rows,2]].__eq__(i[0]))[0]), np.intersect1d(np.where(dataset['bead_idnames'][angle_indices[rows,0]].__eq__(i[0]))[0], np.where(dataset['bead_idnames'][angle_indices[rows,2]].__eq__(i[2]))[0]))
   
    print(sides, i[0],i[2])
    # print(dataset['bead_names'][angle_indices[rows,0]][sides_l])
    # sides_r = np.where(dataset['bead_names'][angle_indices[rows]].__eq__(i[2]))[0]
    # print(np.unique(np.asanyarray(sides_r)) , i[2])
    # print(dataset['bead_names'][angle_indices[rows,2]][sides_r])
    key = str(angle_indices[rows[sides]][0,0]) + "-" +  str(angle_indices[rows[sides]][0,1]) + "-" + str(angle_indices[rows[sides]][0,2])

    if str(i[2] + "-" + i[1] + "-" + i[0]) not in angle_dic:
        angle_dic[i[0] + "-" + i[1] + "-" + i[2]] = np.average(np.asanyarray(angles[key]))




[2.32529865 2.35874371 1.0831449  2.05347879 1.06169008 2.05023157
 2.9510325  1.66981165 1.4915041  2.36495023 1.0714839  2.25291166
 2.06580534 1.0571679  2.08378516 2.96756514 1.75256768 1.79762036
 1.41852567 2.27097708 1.6963522  1.26435567 2.50868473 2.29205144
 2.02753863 1.4661648  2.27495058 2.78251821 1.56877376 1.74795311
 2.05257074 2.00185024 1.66227655 2.49646937 2.0124707  0.76362403
 1.21070422 1.1672644  2.94134065 1.0800928  1.78392697 1.02029549
 1.04120436 1.45342648 1.90776901 2.01293369 1.06737042 2.07669423
 1.05065377 2.08368339]
[ 0  6  7  8 43]
[ 0  5  5  5 27]
[['TYR_SC1' 'TYR_BB' 'TYR_BB']
 ['TYR_BB' 'TYR_BB' 'ASP_BB']
 ['TYR_SC1' 'TYR_BB' 'ASP_BB']
 ['TYR_BB' 'TYR_BB' 'TYR_SC1']
 ['TRP_BB' 'TYR_BB' 'TYR_SC1']]
[0 3] TYR_SC1 TYR_BB
[ 1  2  9 10 11 44 45 46]
[ 1  1  6  6  6 28 28 28]
[['TYR_BB' 'TYR_SC1' 'TYR_SC3']
 ['TYR_SC2' 'TYR_SC1' 'TYR_SC3']
 ['TYR_BB' 'TYR_SC1' 'TYR_SC2']
 ['TYR_SC2' 'TYR_SC1' 'TYR_SC3']
 ['TYR_BB' 'TYR_SC1' 'TYR_SC3']
 ['TYR_BB' 'TYR_

In [ ]:
config_angles_dict = list(zip(angle_dic.keys(), angle_dic.values()))
config_angles = []
for i,j in config_angles_dict:
     config_angles.append("{} : {}".format(i,j))
config_angles


['TYR_SC1-TYR_BB-TYR_BB : 2.325298650052455',
 'TYR_BB-TYR_SC1-TYR_SC2 : 2.364950226745279',
 'TYR_SC2-TYR_SC1-TYR_SC3 : 1.0831448988767551',
 'TYR_BB-TYR_SC1-TYR_SC3 : 2.3587437083537135',
 'TYR_SC1-TYR_SC2-TYR_SC4 : 2.0534787890772694',
 'TYR_SC2-TYR_SC4-TYR_SC3 : 1.061690075435105',
 'TYR_SC1-TYR_SC3-TYR_SC4 : 2.050231574600113',
 'TYR_SC1-TYR_BB-ASP_BB : 1.6698116455026062',
 'TYR_BB-TYR_BB-ASP_BB : 2.9510324991932793',
 'TYR_BB-ASP_BB-ASP_SC1 : 2.680102934226249',
 'ASP_SC1-ASP_BB-PRO_BB : 1.7525676831458727',
 'TYR_BB-ASP_BB-PRO_BB : 2.967565138113218',
 'ASP_BB-PRO_BB-PRO_SC1 : 1.4185256708401917',
 'PRO_SC1-PRO_BB-GLU_BB : 2.270977080790765',
 'ASP_BB-PRO_BB-GLU_BB : 1.7976203569054758',
 'PRO_BB-GLU_BB-GLU_SC1 : 1.2643556685965343',
 'GLU_SC1-GLU_BB-THR_BB : 2.508684732992379',
 'PRO_BB-GLU_BB-THR_BB : 1.696352203062905',
 'GLU_BB-THR_BB-THR_SC1 : 1.4661647999763887',
 'THR_SC1-THR_BB-GLY_BB : 2.0275386281643457',
 'GLU_BB-THR_BB-GLY_BB : 2.2920514359177253',
 'THR_BB-GLY_BB-T

In [ ]:
output_file = "config files/config.naive.angles.yaml"

with open('config files/config.naive.angles.yaml', 'w') as f:
    for line in config_angles:
        f.write(f"{line}\n")
print(f"{output_file} successfully saved!")

config files/config.naive.angles.yaml successfully saved!


In [ ]:
dihedral_conf = ['TRP_BB', 6, 'TYR_BB', 5]
indices = []
bead_dih = []
for bead in dihedral_conf:
    if isinstance(bead, int):
        for index in indices[-1]:
            temp = []
            for i in range(bead):
                temp.append(index + i)
            bead_dih.append(temp)
            
        
        print(bead_dih)
    else:
        indices.append(np.where(bead == dataset['bead_idnames'])[0])
        print(indices[-1])

[21]
[[21, 22, 23, 24, 25, 26]]
[ 0  5 27]
[[21, 22, 23, 24, 25, 26], [0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [27, 28, 29, 30, 31]]


In [ ]:
dihedrals = {}
for residue in bead_dih:
    print(residue)
    if len(residue) == 6:
        residue.pop(0)
        residue.pop(2)
        print(residue)
        poses  = dataset['bead_pos'][:,residue]
        # dihedral.append()
        
    elif len(residue) == 5:
        residue.pop(0)
        residue = np.asanyarray(residue)
        print(residue)
        poses  = dataset['bead_pos'][:,residue]

    
    # print(poses[0])
    plane1 = [poses[:,0], poses[:,1], poses[:,2]]
    plane2 = [poses[:,1], poses[:,2], poses[:,3]]
    # dihedral.append()
    # print(plane1[1], plane2[0])
    vector1 = plane1[0] - plane1[1]
    vector2 = plane1[2] - plane1[1]
    vector3 = plane2[0] - plane2[1]
    vector4 = plane2[2] - plane2[1]

    unit_vec1 = vector1[:] / np.linalg.norm(vector1, axis=-1)[:,None]
    unit_vec2 = vector2[:] / np.linalg.norm(vector2, axis=-1)[:,None]
    unit_vec3 = vector3[:] / np.linalg.norm(vector3, axis=-1)[:,None]
    unit_vec4 = vector4[:] / np.linalg.norm(vector4, axis=-1)[:,None]

    binorm1 = np.cross(unit_vec1, unit_vec2, axis=-1) 
    binorm2 = np.cross(unit_vec3, unit_vec4, axis=-1) 
    binorm1 /=  np.linalg.norm(binorm1, axis=-1)[:,None]
    binorm2 /=  np.linalg.norm(binorm2, axis=-1)[:,None]

    # np.dot(binorm1[:,None], binorm2[:,None])
    # print(unit_vec1[0], unit_vec2[0])
    # print("binorm = ",binorm1.shape, binorm1[0])

    torsion = np.average(np.arccos(np.sum(binorm1[0] * binorm2[0], axis=-1)) - np.pi)
    print(vector1[0], vector2[0], vector3[0],vector4[0])
    dihedrals[str(residue[0]) + "-" + str(residue[1]) + "-" + str(residue[2]) + "-" + str(residue[3])] = torsion
print(dihedrals)


[21, 22, 23, 24, 25, 26]
[22, 23, 25, 26]
[-0.21052643 -0.10127469 -0.07344736] [ 0.16615803 -0.30515071  0.11204046] [-0.16615803  0.30515071 -0.11204046] [-0.19165706 -0.09111921 -0.07048485]
[0, 1, 2, 3, 4]
[1 2 3 4]
[-0.22781514 -0.09302441 -0.05430636] [ 0.16830585 -0.14559914  0.12547652] [-0.16830585  0.14559914 -0.12547652] [-0.23435209 -0.09407391 -0.05366509]
[5, 6, 7, 8, 9]
[6 7 8 9]
[-0.19010701  0.16260578  0.03153797] [ 0.08393348 -0.04469527 -0.23965578] [-0.08393348  0.04469527  0.23965578] [-0.18983577  0.1664407   0.03739517]
[27, 28, 29, 30, 31]
[28 29 30 31]
[-0.01484138  0.12076151 -0.22153118] [-0.20368396 -0.13613173  0.07925915] [ 0.20368396  0.13613173 -0.07925915] [-0.01519755  0.13481999 -0.21627785]
{'22-23-25-26': -3.127234882367652, '1-2-3-4': -3.13057099964908, '6-7-8-9': -3.125491782550143, '28-29-30-31': -3.078785422383152}


In [ ]:
dih_indices = np.asanyarray([list(map(int, item.split('-'))) for item in dihedrals.keys()])
dataset['bead_names'][dih_indices[:,0]]

conf_dihedrals: list = yaml.safe_load(Path("config files/config.naive.per.bead.type.dihedrals.yaml").read_text())

dih_dic = conf_dihedrals
dih_array = np.asanyarray([values for values in dihedrals.values()])
print(dih_array)
for i in dataset['bead_idnames'][dih_indices]:
    rows = np.where(dataset['bead_idnames'][dih_indices[:,0]].__eq__(i[0]))[0]
    print(i,rows,dih_indices[rows,0], sides)
    
    dih_dic[i[0] + "-" + i[1] + "-" + i[2] + "-" + i[3]] = np.mean(np.asanyarray(dih_array[rows]))

dih_dic

[-3.12723488 -3.130571   -3.12549178 -3.07878542]
['TRP_SC1' 'TRP_SC2' 'TRP_SC5' 'TRP_SC4'] [0] [22] [0 1 2]
['TYR_SC1' 'TYR_SC2' 'TYR_SC4' 'TYR_SC3'] [1 2 3] [ 1  6 28] [0 1 2]
['TYR_SC1' 'TYR_SC2' 'TYR_SC4' 'TYR_SC3'] [1 2 3] [ 1  6 28] [0 1 2]
['TYR_SC1' 'TYR_SC2' 'TYR_SC4' 'TYR_SC3'] [1 2 3] [ 1  6 28] [0 1 2]


{'TRP_SC1-TRP_SC2-TRP_SC5-TRP_SC4': -3.127234882367652,
 'TYR_SC1-TYR_SC2-TYR_SC4-TYR_SC3': -3.111616068194125}

In [ ]:
dataset['dihedral_indices'] = dih_indices
dataset['dihedral_indices']

array([[22, 23, 25, 26],
       [ 1,  2,  3,  4],
       [ 6,  7,  8,  9],
       [28, 29, 30, 31]])

In [ ]:
config_dihedrals_per_bead = []
for root_bead in dih_dic:
    config_dihedrals_per_bead.append("{} : {}".format(root_bead, dih_dic[root_bead]))
config_dihedrals_per_bead

['TRP_SC1-TRP_SC2-TRP_SC5-TRP_SC4 : -3.127234882367652',
 'TYR_SC1-TYR_SC2-TYR_SC4-TYR_SC3 : -3.111616068194125']

In [ ]:
output_file = "config files/config.naive.per.bead.type.dihedrals.yaml"

with open('config files/config.naive.per.bead.type.dihedrals.yaml', 'w') as f:
    for line in config_dihedrals_per_bead:
        f.write(f"{line}\n")
print(f"{output_file} successfully saved!")

config files/config.naive.per.bead.type.dihedrals.yaml successfully saved!


In [ ]:
bead_charges = []
for bead in dataset['bead_idnames']:
    if bead == 'GLU_SC1' or bead == 'ASP_SC1' :
        bead_charges.append(-1)
    elif bead == 'ARG_SC2' or bead == 'HIS_SC3' or bead == 'LYS_SC2':
        bead_charges.append(+1)
    else:
        bead_charges.append(0)
print(bead_charges)
dataset['bead_charges'] = np.asanyarray(bead_charges).reshape(-1,1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
beadcharges = list(zip(dataset['bead_idnames'], dataset['bead_charges']))
bead_charge_dict = {}
for i, j in beadcharges:
    bead_charge_dict[i] = j

dataset['bead_charge_dict'] = bead_charge_dict

In [ ]:
config_BeadCharges_per_bead = []
for root_bead in bead_charge_dict:
    config_BeadCharges_per_bead.append("{} : {}".format(root_bead, bead_charge_dict[root_bead][0]))
config_BeadCharges_per_bead

['TYR_BB : 0',
 'TYR_SC1 : 0',
 'TYR_SC2 : 0',
 'TYR_SC4 : 0',
 'TYR_SC3 : 0',
 'ASP_BB : 0',
 'ASP_SC1 : -1',
 'PRO_BB : 0',
 'PRO_SC1 : 0',
 'GLU_BB : 0',
 'GLU_SC1 : -1',
 'THR_BB : 0',
 'THR_SC1 : 0',
 'GLY_BB : 0',
 'TRP_BB : 0',
 'TRP_SC1 : 0',
 'TRP_SC2 : 0',
 'TRP_SC3 : 0',
 'TRP_SC5 : 0',
 'TRP_SC4 : 0']